# Docker Swarm

## 概要

* Docker Swarmとは
    * Dockerが開発するDockerクラスタ管理・運用ツール
* アーキテクチャ
    * dockerコマンドとdockerデーモンのやりとり
        * デフォルトはUNIXドメインソケット(/var/run/docker.sock) 経由で行う
        * ネットワーク(TCP)経由でやり取りを行う事も可能
            * Docker Swarmはこれを利用してリモートのDockerデーモンとやり取りする
    * Swarm Manager
        * 各ノードで動作するDockerデーモンを一括操作する
            * クラスタ上のコンテナ管理を実現する
        * Docker Daemonと同じAPIを実装している
            * ローカルのDockerを利用するのとまったく同じdockerコマンドを使ってコンテナの起動や停止といった管理操作を実行できる
* できること
    * Dockerクラスタを構成するノードから最適なノードを自動で選択
        * ユーザーからのコンテナ作成要求に対し、要求されたコンテナを実行
    * 各ノードで実行されているコンテナの状況を取得
    * コンテナの停止や再起動
    * トラブル対応
        * トラブルで停止した場合などに、停止したコンテナを別のノードで実行させる機能もある
* できないこと
    * コンテナ間のネットワーク的な接続やルーティングについてはDockerが備えている以上の機能は提供されない
    * コンテナイメージの管理についても、Docker Swarmが特別な機能を提供するわけではない
        * Docker HubやDocker registryを利用する

## インストール

### dockerのインストール
ansibleで構築。いずれ追記予定

### Goのインストール

* バージョンは1.7以上
    * CentOS7はyumで入れると1.6が入ってしまうので使えない

```
$ wget https://storage.googleapis.com/golang/go1.8.linux-amd64.tar.gz
$ tar zxvf go1.8.linux-amd64.tar.gz
$ sudo chown root:root -R go
$ sudo mv go /usr/local/
$ sudo vi /etc/profile.d/go.sh
================================================
export GOROOT=/usr/local/go
export PATH=$PATH:$GOROOT/bin
================================================
$ source /etc/profile.d/go.sh

$ mkdir ~/gocode
$ vi .bashrc
================================================
export GOPATH=~/gocode
export PATH=$PATH:$GOPATH/bin
================================================
$ source ~/.bashrc

$ which go
================================================
/usr/local/go/bin/go
================================================

$ go version
================================================
go version go1.8 linux/amd64
================================================
```

### 必要なライブラリのインストール

```
### vndrのインストール
$ go get github.com/LK4d4/vndr

### golintのインストール
$ go get github.com/golang/lint/golint
```

### Swarmリポジトリをcloneする

```
### swarmをclone
$ mkdir -p $GOPATH/src/github.com/docker
$ cd $GOPATH/src/github.com/docker
$ git clone https://github.com/docker/swarm.git
$ cd swarm

### originを自分のリポジトリに指定
$ git remote remove origin
$ git remote add origin https://github.com/ftakao2007/swarm.git
$ git remote add upstream https://github.com/docker/swarm.git
$ git remote set-url --push upstream no-pushing
$ git remote -v
================================================
origin  https://github.com/ftakao2007/swarm.git (fetch)
origin  https://github.com/ftakao2007/swarm.git (push)
upstream        https://github.com/docker/swarm.git (fetch)
upstream        no-pushing (push)
================================================

### 最新を自分のforkに取り込む場合
### $ git fetch upstream master
### $ git rebase upstream/master
### $ git push -f origin master
```

### Swarmバイナリをビルドする

```
### ビルドする
$ cd $GOPATH/src/github.com/docker/swarm
$ go install .

### テスト
$ $GOPATH/bin/swarm -v
================================================
swarm version 1.2.8 (HEAD)
================================================
```

### バイナリを各ホストに配布する

``` 
$ scp $GOPATH/bin/swarm (対象ホスト):
$ ssh (対象ホスト)
$ sudo chown root:root swarm
$ sudo mv swarm /usr/local/bin
$ swarm -version
================================================
swarm version 1.2.8 (HEAD)
================================================
```

### 参考

* [公式手順](https://github.com/docker/swarm/blob/master/CONTRIBUTING.md)
* [Docker SwarmによるDockerクラスタ環境の構築（Dockerの最新機能を使ってみよう：第4回）](http://knowledge.sakura.ad.jp/knowledge/5197/)

### Etcdの設定

* swarm-managerのホストのみ実施

```
### Etcdのインストール
$ sudo yum install etcd

### 外部からのアクセスを受け付けるよう/etc/etcd/etcd.confを修正
### localhostを自分のアドレスに変更
$ cp /etc/etcd/etcd.conf /tmp
$ sudo vi /etc/etcd/etcd.conf
$ diff -u /tmp/etcd.conf /etc/etcd/etcd.conf
================================================
--- /tmp/etcd.conf      2017-07-22 11:35:46.092458672 +0900
+++ /etc/etcd/etcd.conf 2017-07-22 11:36:27.330911397 +0900
@@ -6,7 +6,7 @@
 #ETCD_HEARTBEAT_INTERVAL="100"
 #ETCD_ELECTION_TIMEOUT="1000"
 #ETCD_LISTEN_PEER_URLS="http://localhost:2380"
-ETCD_LISTEN_CLIENT_URLS="http://localhost:2379"
+ETCD_LISTEN_CLIENT_URLS="http://10.136.3.183:2379"
 #ETCD_MAX_SNAPSHOTS="5"
 #ETCD_MAX_WALS="5"
 #ETCD_CORS=""
@@ -17,7 +17,7 @@
 #ETCD_INITIAL_CLUSTER="default=http://localhost:2380"
 #ETCD_INITIAL_CLUSTER_STATE="new"
 #ETCD_INITIAL_CLUSTER_TOKEN="etcd-cluster"
-ETCD_ADVERTISE_CLIENT_URLS="http://localhost:2379"
+ETCD_ADVERTISE_CLIENT_URLS="http://10.136.3.183:2379"
 #ETCD_DISCOVERY=""
 #ETCD_DISCOVERY_SRV=""
 #ETCD_DISCOVERY_FALLBACK="proxy"
================================================
```

* 以下は全ホストで実施

```
### dockerがどの環境設定ファイルやサービス用ファイルを使っているか確認する
### 以下の場合「/usr/lib/systemd/system/docker.service」というサービス用ファイルを使っている
$ systemctl show docker | grep EnvironmentFile
================================================
(何もなし)
================================================

$ systemctl status docker | grep Loaded
================================================
   Loaded: loaded (/usr/lib/systemd/system/docker.service; enabled; vendor preset: disabled)
================================================

$ grep EnvironmentFile /usr/lib/systemd/system/docker.service
================================================
(何もなし)
================================================

### 「/usr/lib/systemd/system/docker.service」にDOCKER_NETWORK_OPTIONSを追加する
### Serviceディレクティブに設定する
$ cp /usr/lib/systemd/system/docker.service /tmp
$ sudo vi /usr/lib/systemd/system/docker.service
$ diff -u /tmp/docker.service /usr/lib/systemd/system/docker.service
================================================
--- /tmp/docker.service 2017-07-22 12:04:17.007548515 +0900
+++ /usr/lib/systemd/system/docker.service      2017-07-22 12:06:11.246932774 +0900
@@ -5,6 +5,7 @@
 Wants=network-online.target

 [Service]
+EnvironmentFile=-/etc/sysconfig/docker-network
 Type=notify
 # the default is not to use systemd for cgroups because the delegate issues still
 # exists and systemd currently does not support the cgroup feature set required
================================================

### 「--cluster-store」はEtcdが動いているホストのIP、「--cluster-advertise」は各ホストのIP
$ sudo vi /etc/sysconfig/docker-network
================================================
DOCKER_NETWORK_OPTIONS='--cluster-store=etcd://10.136.3.183:2379 --cluster-advertise=10.136.3.184:2376'
================================================

### デーモンをreload
$ sudo systemctl daemon-reload

### dockerを再起動
$ sudo systemctl restart docker.service

### 設定の確認
$ systemctl show docker | grep EnvironmentFile
================================================
EnvironmentFile=/etc/sysconfig/docker-network (ignore_errors=yes)
================================================

### Etcdをインストールしているホストに接続できるかcurlで確認
$ curl -L http://10.136.3.183:2379/version
================================================
{"etcdserver":"3.1.9","etcdcluster":"3.1.0"}
================================================
```

## Swarm Managerの起動と接続確認

### Swarm Managerを起動する

```
### tcp://＜ホストのIPアドレス＞:3375 etcd://＜Etcdを稼働させているホストのIPアドレス＞:2379
### Swarm ManagerのホストにEtcdをインストールすることがほとんどなので通常は同じアドレスになるはず
$ swarm manage -H tcp://10.136.3.183:3375 etcd://10.136.3.183:2379
================================================
INFO[0000] Initializing discovery without TLS
INFO[0000] Listening for HTTP                            addr=10.136.3.183:3375 proto=tcp
================================================
※ 検証のため手で実行しているが、運用するときはデーモン化しておく

```



### 各ホストから接続確認

```
### 「--advertise」は自ホストのIP、「etcd」はetcdが起動しているホストのID
swarm join --advertise=10.136.3.184:2375 etcd://10.136.3.183:2379
================================================
INFO[0000] Initializing discovery without TLS
INFO[0000] Registering on the discovery service every 1m0s...  addr=10.136.3.184:2375 discovery=etcd://10.136.3.183:2379
INFO[0060] Registering on the discovery service every 1m0s...  addr=10.136.3.184:2375 discovery=etcd://10.136.3.183:2379
================================================
※ 検証のため手で実行しているが、運用するときはデーモン化しておく

### 接続確認
### IPアドレスはSwarm Managerのホストを指定
$ docker -H tcp://10.136.3.183:3375 info
================================================
Containers: 0
 Running: 0
 Paused: 0
 Stopped: 0
Images: 0
Server Version: swarm/1.2.8
Role: primary
Strategy: spread
Filters: health, port, containerslots, dependency, affinity, constraint, whitelist
Nodes: 1
 (unknown): 10.136.3.184:2375
  m ID:
  m Status: Pending
  m Containers: 0
  m Reserved CPUs: 0 / 0
  m Reserved Memory: 0 B / 0 B
  m Labels:
  m Error: Cannot connect to the Docker daemon at tcp://10.136.3.184:2375. Is the docker daemon running?
  m UpdatedAt: 2017-07-22T03:34:09Z
  m ServerVersion:
Plugins:
 Volume:
 Network:
Swarm:
 NodeID:
 Is Manager: false
 Node Address:
Kernel Version: 3.10.0-514.26.2.el7.x86_64
Operating System: linux
Architecture: amd64
CPUs: 0
Total Memory: 0B
Name: kubernetes-master
Docker Root Dir:
Debug Mode (client): false
Debug Mode (server): false
Experimental: false
Live Restore Enabled: false

WARNING: No kernel memory limit support
================================================


```